In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


dates = pd.date_range("2024-01-01", periods=60, freq="B")
tickers = ["AAPL", "GOOG", "TSLA"]

data = {
    "date": np.repeat(dates, len(tickers)),
    "ticker": tickers * len(dates),
    "price": np.random.lognormal(mean=5, sigma=0.05, size=len(dates) * len(tickers)),
    "volume": np.random.randint(1e4, 1e5, size=len(dates) * len(tickers)),
}
df = pd.DataFrame(data)
df["price"] = df["price"].round(2)

print("Первые строки:")
print(df.head())
print("\nИнфо о датафрейме:")
print(df.info())
print("\nОписательная статистика по числовым колонкам:")
print(df.describe())

In [ ]:
df = df.set_index("date").sort_index()

tsla_big = df[(df["ticker"] == "TSLA") & (df["volume"] > 80000)]
print(tsla_big.head())

print(df[["ticker", "price"]].head())

print(df.sort_values("price", ascending=False).head())


In [ ]:
#средняя цена и объём по тикеру
group_ticker = df.groupby("ticker").agg(
    mean_price=("price", "mean"),
    max_price=("price", "max"),
    mean_volume=("volume", "mean"),
)
print(group_ticker)

#ежедневный оборот (цена * объём) по тикеру
df["turnover"] = df["price"] * df["volume"]
daily_turnover = df.groupby(["date", "ticker"])["turnover"].sum().unstack("ticker")
print("\nЕжедневный оборот (первые строки):")
print(daily_turnover.head())


In [ ]:
#работа с пропусками (искусственно создадим пропуски)

df_na = df.copy()
df_na.loc[df_na.sample(frac=0.05, random_state=0).index, "price"] = np.nan
print("\nКоличество пропусков по колонкам:")
print(df_na.isna().sum())

# Заполнение пропусков вперёд по каждому тикеру
df_na["price_filled"] = df_na.groupby("ticker")["price"].ffill()
print("\nПосле заполнения пропусков (price_filled):")
print(df_na[["ticker", "price", "price_filled"]].head(10))


In [ ]:

plt.style.use("seaborn-v0_8") 

#линейный график: цена AAPL во времени
aapl = df[df["ticker"] == "AAPL"]
plt.figure(figsize=(10, 4))
plt.plot(aapl.index, aapl["price"], marker="o", linewidth=1)
plt.title("Цена AAPL во времени")
plt.xlabel("Дата")
plt.ylabel("Цена")
plt.grid(True)
plt.tight_layout()
plt.show()

#точечная диаграмма: зависимость цены от объёма для TSLA
tsla = df[df["ticker"] == "TSLA"]
plt.figure(figsize=(6, 4))
plt.scatter(tsla["volume"], tsla["price"], alpha=0.7)
plt.title("TSLA: цена vs объём")
plt.xlabel("Объём")
plt.ylabel("Цена")
plt.tight_layout()
plt.show()

#cтолбчатый график: средняя цена по тикерам
plt.figure(figsize=(6, 4))
plt.bar(group_ticker.index, group_ticker["mean_price"])
plt.title("Средняя цена по тикерам")
plt.xlabel("Тикер")
plt.ylabel("Средняя цена")
plt.tight_layout()
plt.show()

#гистограмма: распределение объёма
plt.figure(figsize=(6, 4))
plt.hist(df["volume"], bins=20, edgecolor="black")
plt.title("Распределение объёма сделок")
plt.xlabel("Объём")
plt.ylabel("Частота")
plt.tight_layout()
plt.show()
